# Morbilidad Hospitalaria España 2016-2019
## Creación dataframe para visualización

[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [1]:
import pandas as pd
import datetime

In [2]:
#tenemos 4 dataframes por años desde 2016 a 2019
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [3]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [4]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
808959,4416704,7,2,7,1,9052016,G43901,1,64,4,2,1,1.06965
79027,23690607,28,2,28,2,17062016,A870,1,1,6,11,5,1.00000
862013,38887148,43,1,43,1,9052018,G4733,1,69,11,3,1,1.04346
781788,14644613,13,2,13,1,3032017,F5002,1,17,6,25,4,1.00000
533895,25376799,28,2,28,1,8112019,D271,1,51,4,3,1,1.01201


In [5]:
#el código de las enferemades mentales empeizan por F (hacemos una selección)
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [6]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [7]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos para tener los tipos de enfermedasdes mentales agrupadas

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [8]:
morbilidad["code"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [9]:
morbilidad.code.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

## Añadimos códigos CIE (Capítulos y diagnósticos)

In [10]:
#importamos la tabla de correspondencia entre código de diagnóstico agrupadas y nombre
diagnos = pd.read_csv("../data/diagnos.csv")

In [11]:
diagnos.sample(3) 

,code,diag,cap,title
30,F32,"Trastorno depresivo mayor, episodio único",F30-39,"Trastornos del estado de animo, trastornos afe..."
41,F50,Trastornos de la ingestión de alimentos,F50-59,Síndromes del comportamiento asociados con alt...
74,F98,Otros trastornos emocionales y de comportamien...,F90-F98,Trastornos emocionales y del comportamiento qu...


In [12]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [13]:
dicc_cap = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_cap[r.code] = r.cap

In [14]:
morbilidad["cap"] = morbilidad.code.map(dicc_cap) #creamos nueva columnas

In [15]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap
670641,1,2,1,2,13102016,F1020,1,48,8,1.00000,F10,F10-F19
706997,2,2,2,1,24082018,F200,1,48,3,1.00000,F20,F20-29
714130,8,1,8,2,23062016,F29,1,40,8,1.02915,F29,F20-29


In [16]:
discapacidad = morbilidad[morbilidad["cap"] == 'F70-79'].index #dropeamos el tipo de retraso mental
morbilidad.drop(discapacidad,inplace = True)

## Creamos fecha de ingreso

In [17]:
#creamos fecha a través de FxAlta
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [18]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [19]:
#creamos fecha ingreso restando a fecha_al el número de día ingresado (DiasEstancia)
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [20]:
morbilidad.drop(['FxAlta'],axis=1,inplace = True)

In [21]:
# Cambiamos algunos datos categóricos que vienen como números 

In [22]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,fecha_al,fecha_ing
772614,28,2,28,1,F42,1,60,11,1.00000,F42,F40-49,2017-04-28,2017-04-17
689741,20,2,20,1,F200,1,48,287,1.00000,F20,F20-29,2016-11-23,2016-02-10
688292,8,2,8,1,F200,1,67,1,1.01123,F20,F20-29,2016-11-23,2016-11-22


In [23]:
#diccionarios códigos
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}
num_str = {'1':'01',  '2':'02',  '3':'03',  '4':'04',  '5':'05',  '6':'06',  '7':'07',  '8':'08',  '9':'09'}

In [24]:
morbilidad.ProvHosp = morbilidad.ProvHosp.astype('string').replace(num_str)
morbilidad.ProvResi = morbilidad.ProvResi.astype('string').replace(num_str)

In [25]:
morbilidad['Entrada'] = morbilidad.DiagEntr.map(dic_entrada)
morbilidad['Alta'] = morbilidad.MotivoAlta.map(dic_alta)
morbilidad['SexNom'] = morbilidad.Sexo.map(dic_sexo)

In [27]:
morbilidad.to_csv("../data/morbilidad_Fgen.csv",index=False)